In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%matplotlib inline
from matplotlib import*
import warnings
warnings.filterwarnings('ignore')
from sklearn.datasets import *
from IPython.display import Image, display_svg
import re
import random
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn import datasets, linear_model
from os import path
import ipywidgets as widgets
import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from scipy.cluster.vq import kmeans, vq , whiten
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import scipy as sp
import ipywidgets as widgets
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from copy import deepcopy
from sklearn import tree
import collections
import h2o
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy import stats
from imblearn.over_sampling import SMOTE, ADASYN
from collections import Counter
from sklearn.svm import LinearSVC
from xgboost import plot_importance
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn.ensemble import AdaBoostClassifier
from mlxtend.classifier import StackingCVClassifier
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
# h2o.init()

# Drop Duplicates

In [2]:
df = pd.read_csv('C:\\Users\\abhin\\Desktop\\M571\\bankruptcy_Train.csv')
df.drop_duplicates(keep = "first", inplace = True)

# Up Sample

In [3]:
x1, y1 = df.drop("class", axis=1), df["class"]

x_resampled, y_resampled = SMOTE().fit_resample(x1, y1)
x_resampled = pd.DataFrame(x_resampled)
features = ['Attr1', 'Attr2', 'Attr3', 'Attr4', 'Attr5', 'Attr6', 'Attr7', 'Attr8',
       'Attr9', 'Attr10', 'Attr11', 'Attr12', 'Attr13', 'Attr14', 'Attr15',
       'Attr16', 'Attr17', 'Attr18', 'Attr19', 'Attr20', 'Attr21', 'Attr22',
       'Attr23', 'Attr24', 'Attr25', 'Attr26', 'Attr27', 'Attr28', 'Attr29',
       'Attr30', 'Attr31', 'Attr32', 'Attr33', 'Attr34', 'Attr35', 'Attr36',
       'Attr37', 'Attr38', 'Attr39', 'Attr40', 'Attr41', 'Attr42', 'Attr43',
       'Attr44', 'Attr45', 'Attr46', 'Attr47', 'Attr48', 'Attr49', 'Attr50',
       'Attr51', 'Attr52', 'Attr53', 'Attr54', 'Attr55', 'Attr56', 'Attr57',
       'Attr58', 'Attr59', 'Attr60', 'Attr61', 'Attr62', 'Attr63', 'Attr64']
x_resampled.columns = features
y_resampled = pd.DataFrame(y_resampled)
y_resampled.columns = ["class"]

# Create train and validation set

In [4]:
# x1, y1 = df.drop("class", axis=1), df["class"]
x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=12345)
eval_set = [(x_train, y_train)]
eval_metric = ["auc"]

# Run baseline XGBoost

In [5]:
xg_reg = XGBClassifier()
xg_reg.fit(x_train, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=False)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

# Gradient Boost

In [6]:
gbc = GradientBoostingClassifier()
gbc.fit(x_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

# AdaBoost

In [7]:
abc = AdaBoostClassifier()
abc.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)

# Light GBM

In [8]:
lgbm = lgb.LGBMClassifier()
lgbm.fit(x_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

# Hyper Parameter Tuning

In [9]:
#XGBoost
params = {"learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth": [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight": [ 1, 3, 5, 7 ],
 "gamma": [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree": [ 0.3, 0.4, 0.5 , 0.7 ] }
folds = 3
param_comb = 40
data_dmatrix = xgb.DMatrix(data=x_train,label=y_train)
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV(xg_reg, param_distributions=params, n_iter=param_comb, scoring='roc_auc',
                                   n_jobs=4, cv=skf.split(x_train,y_train), verbose=10, random_state=1001 )
random_search.fit(x_train, y_train)
print('\n Best estimator:')
print(random_search.best_estimator_)

Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   18.0s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   23.9s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:   46.8s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   58.3s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  1.5min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.9min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  2.2min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  2.6min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:  3.6min
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:  4.0min
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:  4.4min finished



 Best estimator:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0.0,
              learning_rate=0.2, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [10]:
# lgbm tune
def learning_rate_010_decay_power_099(current_iter):
    base_learning_rate = 0.1
    lr = base_learning_rate  * np.power(.99, current_iter)
    return lr if lr > 1e-3 else 1e-3

def learning_rate_010_decay_power_0995(current_iter):
    base_learning_rate = 0.1
    lr = base_learning_rate  * np.power(.995, current_iter)
    return lr if lr > 1e-3 else 1e-3

def learning_rate_005_decay_power_099(current_iter):
    base_learning_rate = 0.05
    lr = base_learning_rate  * np.power(.99, current_iter)
    return lr if lr > 1e-3 else 1e-3

fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'auc', 
            "eval_set" : [(x_test,y_test)],
            'eval_names': ['valid'],
            #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
            'verbose': 100,
            'categorical_feature': 'auto'}

from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

#This parameter defines the number of HP points to be tested
n_HP_points_to_test = 100

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
skf = StratifiedKFold(n_splits=3, shuffle = True, random_state = 1001)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=skf.split(x_train,y_train),
    refit=True,
    random_state=314,
    verbose=10, n_jobs=-1)

#This parameter defines the number of HP points to be tested
n_HP_points_to_test = 100

gs.fit(x_train, y_train, **fit_params)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   44.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   54.3s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3

Training until validation scores don't improve for 30 rounds
[100]	valid's auc: 0.99994
[200]	valid's auc: 0.999976
Early stopping, best iteration is:
[215]	valid's auc: 0.999979
Best score reached: 0.9999380629446257 with params: {'colsample_bytree': 0.952164731370897, 'min_child_samples': 111, 'min_child_weight': 0.01, 'num_leaves': 38, 'reg_alpha': 0, 'reg_lambda': 0.1, 'subsample': 0.3029313662262354} 


In [11]:
#gbm tune

parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10, 40, 70, 100, 130, 160, 190]
    }
n_HP_points_to_test = 25
skf = StratifiedKFold(n_splits=3, shuffle = True, random_state = 1001)
gclf = GradientBoostingClassifier(max_depth=-1, random_state=314, n_estimators=5000)
gbtune = RandomizedSearchCV(
    estimator=gclf, param_distributions=parameters, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=skf.split(x_train,y_train),
    refit=True,
    random_state=314,
    verbose=10, n_jobs=-1)

gbtune.fit(x_train, y_train)
print('Best score reached: {} with params: {} '.format(gbtune.best_score_, gbtune.best_params_))

Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 16.1min
[Parallel(n_jobs=-1)]: Done  68 out of  75 | elapsed: 16.4min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 20.2min finished


Best score reached: 0.9449882972981684 with params: {'subsample': 0.95, 'n_estimators': 130, 'min_samples_split': 0.46363636363636374, 'min_samples_leaf': 0.24545454545454548, 'max_features': 'log2', 'max_depth': 8, 'loss': 'deviance', 'learning_rate': 0.15, 'criterion': 'friedman_mse'} 


In [12]:
#ada tune
param_dist = {
 'n_estimators': [50, 100, 200, 300],
 'learning_rate' : [0.01,0.05,0.1,0.3,1]
 }
n_HP_points_to_test = 40

skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 1001)
DTC = DecisionTreeClassifier(random_state = 11, max_features = "auto", class_weight = "balanced",max_depth = None)
ada = AdaBoostClassifier(base_estimator = DTC)
abtune = RandomizedSearchCV(
    estimator=ada, param_distributions=param_dist, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=skf.split(x_train,y_train),
    refit=True,
    random_state=314,
    verbose=10, n_jobs=-1)

abtune.fit(x_train, y_train)
print('Best score reached: {} with params: {} '.format(abtune.best_score_, abtune.best_params_))

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  96 out of 100 | elapsed:    3.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.3s finished


Best score reached: 0.9553835297776863 with params: {'n_estimators': 300, 'learning_rate': 0.05} 


# Tuned Models

In [9]:
# XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.8, gamma=0.0,
#               learning_rate=0.11, max_delta_step=0.5, max_depth=3,
#               min_child_weight=2, missing=None, n_estimators=300, n_jobs=-1,
#               objective='binary:logistic', random_state=0,
#               reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1, seed=None,
#               silent=False, subsample=0.8, verbosity=1)
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0.0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
xgb.fit(x_train, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=False)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0.0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [10]:
# lgb.LGBMClassifier(colsample_bytree = 0.952164731370897, min_child_samples= 111, 
#                           min_child_weight= 0.01,num_leaves= 38, reg_alpha= 0, reg_lambda= 0.1, 
#                           subsample = 0.3029313662262354)
lgbm = lgb.LGBMClassifier(colsample_bytree = 0.952164731370897, min_child_samples = 111, 
        min_child_weight = 0.01, num_leaves = 38, reg_alpha = 0, 
        reg_lambda = 0.1, subsample = 0.3029313662262354)
lgbm.fit(x_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.952164731370897, importance_type='split',
               learning_rate=0.1, max_depth=-1, min_child_samples=111,
               min_child_weight=0.01, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=38, objective=None, random_state=None,
               reg_alpha=0, reg_lambda=0.1, silent=True,
               subsample=0.3029313662262354, subsample_for_bin=200000,
               subsample_freq=0)

In [11]:
gbc = GradientBoostingClassifier(subsample= 0.95, n_estimators= 130, 
                                 min_samples_split= 0.46363636363636374, min_samples_leaf= 0.24545454545454548, 
                                 max_features= 'log2', max_depth= 8, loss= 'deviance', 
                                 learning_rate= 0.15, criterion= 'friedman_mse')
gbc.fit(x_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.15, loss='deviance', max_depth=8,
                           max_features='log2', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=0.24545454545454548,
                           min_samples_split=0.46363636363636374,
                           min_weight_fraction_leaf=0.0, n_estimators=130,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=0.95, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [12]:
abc = AdaBoostClassifier(n_estimators= 300, learning_rate= 0.01)
abc.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.01,
                   n_estimators=300, random_state=None)

# Stacked Model

In [13]:
# Stack up all the models above, optimized using xgboost
stack_gen = StackingCVClassifier(classifiers=(xgb, abc, gbc, lgbm),
                                meta_classifier=xgb,
                                use_features_in_secondary=True)
stack_gen.fit(np.array(x_train), np.array(y_train))

StackingCVClassifier(classifiers=(XGBClassifier(base_score=0.5,
                                                booster='gbtree',
                                                colsample_bylevel=1,
                                                colsample_bynode=1,
                                                colsample_bytree=0.4, gamma=0.0,
                                                learning_rate=0.1,
                                                max_delta_step=0, max_depth=3,
                                                min_child_weight=1,
                                                missing=None, n_estimators=300,
                                                n_jobs=1, nthread=None,
                                                objective='binary:logistic',
                                                random_state=0, reg_alpha=0,
                                                reg_lambda=1,
                                                scale_pos_weig...
    

# Blend

In [14]:
def rmsle(y, y_pred):
    return roc_auc_score(y, y_pred)

def blended_predictions(x_test):
    return ((0.25 * xgb.predict_proba(x_test)) + 
            (0.2 * gbc.predict_proba(x_test)) + 
            (0.05 * abc.predict_proba(x_test)) +
            (0.25 * lgbm.predict_proba(x_test)) +
            (0.25 * stack_gen.predict_proba(np.array(x_test))))

In [15]:
blend = blended_predictions(x_test)
blend = ((0.25 * xgb.predict(x_test)) + 
            (0.2 * gbc.predict(x_test)) + 
            (0.05 * abc.predict(x_test)) +
            (0.25 * lgbm.predict(x_test)) +
            (0.25 * stack_gen.predict(np.array(x_test))))
# pd.DataFrame(blend)
blended_score = rmsle(y_test, blend)
blended_score

0.9971154379541798

In [23]:
# roc_auc_score(y_test, preds[1])

In [20]:
test=pd.read_csv('C:\\Users\\abhin\\Desktop\\M571\\bankruptcy_Test_X.csv')
test1 = test.drop("ID", axis=1)
pred_final = pd.DataFrame(blended_predictions(test1))
pred_final = pd.concat([test["ID"], pred_final[1]], axis = 1)
pd.DataFrame(pred_final).to_csv('C:\\Users\\abhin\\Desktop\\M571\\d4b_10_2_v4.csv')
pred_final.head()

,ID,1
0,1,0.274413
1,2,0.022292
2,3,0.011324
3,4,0.101500
4,5,0.147457


In [ ]:
# pred_val = xgb.predict(x_test)
# roc_auc_score(y_test, pred_val)
# pred_val1 = gbc.predict(x_test1)
# roc_auc_score(y_test1, pred_val)
# pred_val2 = abc.predict(x_test1)
# roc_auc_score(y_test1, pred_val)
# pred = xgb.predict(x_test)
# roc_auc_score(y_test, pred)

In [ ]:
# test=pd.read_csv('C:\\Users\\abhin\\Desktop\\M571\\bankruptcy_Test_X.csv')
# test1 = test.drop("ID", axis=1)
# pred_final = xgb.predict_proba(test1)
# pred_final = pd.DataFrame(pred_final)
# pred_final = pd.concat([test["ID"], pred_final[1]], axis = 1)
# pred_final.head()
# pd.DataFrame(pred_final).to_csv('C:\\Users\\abhin\\Desktop\\M571\\d4b_10_2_v2.csv')

In [ ]:
# # drop outliers
# z = np.abs(stats.zscore(temp))
# temp = temp[(z < 4).all(axis=1)]
# temp.shape

# correlated_features = set()
# correlation_matrix = temp.corr()
# for i in range(len(correlation_matrix .columns)):
#     for j in range(i):
#         if abs(correlation_matrix.iloc[i, j]) > 0.8:
#             colname = correlation_matrix.columns[i]
#             correlated_features.add(colname)

# temp.drop(labels=correlated_features, axis=1, inplace=True)

#separate target and predictors
# boruta = ["Attr2","Attr5","Attr6","Attr9","Attr12","Attr13","Attr16","Attr17","Attr19","Attr23","Attr25",
#           "Attr26","Attr27","Attr29","Attr34","Attr35","Attr39","Attr46"]
# x_resampled = x_resampled[boruta]
# xg_reg = xgb.XGBClassifier(objective ='binary:logistic',alpha = 10, silent=False, 
#                       scale_pos_weight=1, colsample_bynode=1, colsample_bylevel=1,
#                       learning_rate=0.12,  
#                       colsample_bytree = 0.8,
#                       subsample = 0.8, 
#                       n_estimators=1000, 
#                       reg_alpha = 0.3,
#                       max_depth=15, 
#                       gamma=0.5)
# xg_reg = XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.8, gamma=0.0,
#               learning_rate=0.11, max_delta_step=0.5, max_depth=15,
#               min_child_weight=2, missing=None, n_estimators=300, n_jobs=1,
#               objective='binary:logistic', random_state=0,
#               reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1, seed=None,
#               silent=False, subsample=0.8, verbosity=1)
# xg_reg = XGBClassifier(alpha=0.07, base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.8, gamma=0.0,
#               learning_rate=0.07, max_delta_step=0,
#               max_depth=15, min_child_weight=1.0, missing=None,
#               n_estimators=1000, n_jobs=1, nthread=None,
#               objective='binary:logistic', random_state=0, reg_alpha=0.3,
#               reg_lambda=1, scale_pos_weight=1, seed=None, silent=False,
#               subsample=0.8, tree_method='hist', grow_policy='lossguide', max_leaves=50, verbosity=1)
# xg_reg = XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.6,
#               gamma=1.4, learning_rate=0.4, max_delta_step=0.5,
#               max_depth=18, min_child_weight=1.0, missing=None,
#               n_estimators=350, n_jobs=4, nthread=None,
#               objective='binary:logistic', random_state=0, reg_alpha=0.3,
#               reg_lambda=1, scale_pos_weight=1, seed=None, silent=False,
#               subsample=0.8, verbosity=1)
# xg_reg = XGBClassifier(alpha=1, base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.7, gamma=2,
#               learning_rate=0.1, max_delta_step=1.5, max_depth=17,
#               min_child_weight=1.0, missing=None, n_estimators=300, n_jobs=1,
#               nthread=None, objective='binary:logistic', random_state=0,
#               reg_lambda=1, scale_pos_weight=1, seed=None,
#               silent=False, subsample=0.6, verbosity=1)
# xg_reg = XGBClassifier(alpha=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=1, gamma=0.0, reg_lambda=0,
#               learning_rate=0.11, max_delta_step=0.5, max_depth=2,
#               min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
#               nthread=None, objective='binary:logistic', random_state=0,
#               scale_pos_weight=1, seed=None,
#               silent=None, subsample=1, verbosity=1)
# xg_reg = XGBClassifier(alpha=2.5, base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=1, gamma=0.5, reg_lambda=4.0,
#               learning_rate=0.11, max_delta_step=0.0,
#               max_depth=2, min_child_weight=1, missing=None, n_estimators=300,
#               n_jobs=1, nthread=None, objective='binary:logistic',
#               random_state=0,scale_pos_weight=1,
#               seed=None, silent=None, subsample=1, verbosity=1)
# params = {
#         'min_child_weight': np.arange(0,11,0.5),
#         'gamma': np.arange(0,2,0.2),
#         'subsample': np.arange(0,1,0.2),
#         'colsample_bytree': np.arange(0,1,0.2),
#         'max_depth': np.arange(0,27),
#         "learning_rate": np.arange(0,0.3,0.01)
#         }
# params = {
#         'n_estimators': np.arange(200,600,50),
#         'min_child_weight': np.arange(0,7,0.5),
#         'gamma': np.arange(0,5,0.2),
#         'subsample': np.arange(0,1,0.2),
#         'colsample_bytree': np.arange(0,1,0.2),
#         'max_depth': np.arange(0,27,2),
#         "learning_rate": np.arange(0,2,0.2)
#         }
# boruta = ["Attr2","Attr5","Attr6","Attr9","Attr12","Attr13","Attr16","Attr17","Attr19","Attr23","Attr25",
#           "Attr26","Attr27","Attr29","Attr34","Attr35","Attr39","Attr46"]
# imp_features = ['Attr27','Attr5','Attr41','Attr13','Attr37','Attr9','Attr34','Attr6','Attr39','Attr58','Attr26',
#                 'Attr46','Attr60','Attr24','Attr9','Attr55','Attr59','Attr25','Attr61','Attr30','Attr1','Attr29']
# # load X and y
# x, y = df.drop('class',axis=1),df['class']
# # define random forest classifier, with utilising all cores and
# # sampling in proportion to y labels
# forest = RandomForestClassifier(n_jobs=4, class_weight='balanced')
 
# # define Boruta feature selection method
# feat_selector = BorutaPy(forest, n_estimators='auto', verbose=2)
 
# # find all relevant features
# feat_selector.fit(x.values, y.values)
# feat_selector.support_